In [1]:
import os
import numpy as np
import pandas as pd
from math import ceil
from collections import deque
from itertools import combinations
from collections import defaultdict
import math
import sys
global maxdays
current_directory = os.getcwd()
%cd \Users\\Sebas\\OneDrive\\Documenten\\School\\3e jaar\\Project CO


C:\Users\Sebas\OneDrive\Documenten\School\3e jaar\Project CO


In [2]:
directory = "\\Users\\Sebas\\OneDrive\\Documenten\\School\\3e jaar\\Project CO"
filename = 'CO_Case2418.txt'
output_file_name = "CO_Case2418_sol.txt"

"""
 Read the instance and assign variables to the different values
"""

data = pd.read_csv(filename, sep='\t+', header=None, engine='python')
data = data[0].str.split(expand=True)
title = data.iloc[1,2] + ' ' + data.iloc[1,3]
maxdays = data.iloc[2,2]
maxdays = int(maxdays)
constraints = data[[0, 2]].iloc[2:5]
constraints = constraints.set_index(0)[2].to_dict()
constraints = dict((k, int(v)) for k, v in constraints.items())
costs = data[[0, 2]].iloc[5:11]
costs = costs.set_index(0)[2].to_dict()
costs = dict((k, int(v)) for k, v in costs.items())
numberMachines = int(data.iloc[11][2])
index = (11 + numberMachines + 1)  # extra step that ensures the right values are taken
machines = np.array(data[[0, 1, 2]][12:index].astype(int))
numberLocations = int(data.iloc[index][2])
locations = np.array(data[[0, 1, 2]][index + 1:(index + numberLocations + 1)].astype(int))
index += numberLocations + 1  # extra step that ensures the right values are taken
depot = locations[0]
numberRequests = int(data.iloc[index][2])
requests = np.array(data[[0, 1, 2, 3, 4, 5]][index + 1:(index + numberRequests + 1)].astype(int))
index += numberRequests + 1  # extra step that ensures the right values are taken
numberTechnicians = int(data.iloc[index][2])
technicians = np.array(
        data[[i for i in range(0, numberMachines + 4)]][index + 1:(index + numberTechnicians + 1)].astype(int))
constraints

DAYS = constraints['DAYS']
TRUCK_CAPACITY = constraints['TRUCK_CAPACITY']
TRUCK_MAX_DISTANCE = constraints['TRUCK_MAX_DISTANCE']

TRUCK_DISTANCE_COST = costs['TRUCK_DISTANCE_COST']
TRUCK_DAY_COST = costs['TRUCK_DAY_COST']
TRUCK_COST = costs['TRUCK_COST']
TECHNICIAN_DISTANCE_COST = costs['TECHNICIAN_DISTANCE_COST']
TECHNICIAN_DAY_COST = costs['TECHNICIAN_DAY_COST']
TECHNICIAN_COST = costs['TECHNICIAN_COST']
IDLE_MACHINE_COSTS = {machine[0]: machine[2] for machine in machines}


In [3]:
def nearest_neighbor(point, unvisited_points, locations):
    #Find the nearest neighbor of a given point among a list of unvisited points. Returns nearest_point: The nearest neighbor of the given point.
    min_distance = float('inf')
    nearest_point = None
    point_coordinates = get_location_coordinates(point, locations)
    
    for candidate_point in unvisited_points:
        candidate_point_id = int(candidate_point)
        if candidate_point_id != int(point):
            candidate_coordinates = get_location_coordinates(candidate_point_id, locations)
            dist = calculate_distance(point_coordinates, candidate_coordinates)
            if dist is not None and dist < min_distance:
                min_distance = dist
                nearest_point = candidate_point_id
    return nearest_point

def get_location_coordinates(location_id, locations):
    #Get the coordinates of a location based on its ID. Returns coordinates: Tuple (x, y) representing the coordinates of the location.

    for location in locations:
        if location[0] == location_id:
            return (location[1], location[2])
    return None

def calculate_distance(coordinates1, coordinates2):
    #Calculate the Euclidean distance between two points given their coordinates and round up to the nearest integer using the ceil function. Returns
    #distance: The Euclidean distance between the two points, rounded up to the nearest integer.
    x1, y1 = coordinates1
    x2, y2 = coordinates2
    distance = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    return ceil(distance)

def get_location_details(requests, locations, request_id):
    #Get the location details (ID and coordinates) corresponding to the given request ID.Returns location_details as a tuple 
    #[(location_id, x_coordinate, y_coordinate)]
  
    for request in requests:
        if request[0] == request_id:  
            location_id = request[1]
            for location in locations:
                if location[0] == location_id:  
                    return (location_id, location[1], location[2])  
    return None  

def calculate_request_distance(requests, locations, request_id1, request_id2):
    #Calculate the distance between two requests based on their corresponding location coordinates. Returns the Euclidean distance.

    location1 = get_location_details(requests, locations, request_id1)
    location2 = get_location_details(requests, locations, request_id2)
    
    if location1 and location2:
        coordinates1 = (location1[1], location1[2])
        coordinates2 = (location2[1], location2[2])
        distance = calculate_distance(coordinates1, coordinates2)
        return distance
    else:
        return None

def get_requests_for_location(requests, location_id):
    #Get the list of request IDs corresponding to the given location ID. 

    request_ids = [req[0] for req in requests if req[1] == location_id]
    return request_ids

unique_location_ids = set(req[1] for req in requests)

for location_id in unique_location_ids:
    request_ids_for_location = get_requests_for_location(requests, location_id)
  

In [4]:
def get_machine_type_from_request(requests):
    # Get the machine type from each request and store it in a dictionary where keys are request IDs and values are machine types.
    machine_types = {}
    for request in requests:
        request_id = request[0]
        machine_type = request[4]
        machine_types[request_id] = machine_type

    return machine_types

machine_types = get_machine_type_from_request(requests)

def get_machine_count_from_request(requests):
    machine_numbers = {}
    for request in requests:
        request_id = request[0]
        machine_number = request[5]
        machine_numbers[request_id] = machine_number

    return machine_numbers

machine_numbers = get_machine_count_from_request(requests)

def get_penalty_cost_from_machine(machines):
    # Extract the penalty cost from each request. Returns dictionary where keys are request IDs and values are the penalty costs.
    penalty_cost = {}
    for mach in machines:
        penalty_cost[mach[0]] = mach[2]
    return penalty_cost

penalty_costs_machines = get_penalty_cost_from_machine(machines)


In [5]:
def calculate_depot_to_request_distance(locations, depot_loc_ID, request_id):
    #Calculate the Euclidean distance between the depot and a specific request based on their corresponding location coordinates.

    depot_coordinates = get_location_coordinates(depot_loc_ID, locations)
    request_location_id = [request[1] for request in requests if request[0] == request_id]
    if request_location_id:
        request_coordinates = get_location_coordinates(request_location_id[0], locations)
        distance = calculate_distance(depot_coordinates, request_coordinates)
        return distance
    else:
        return None

# Calculate the distances between the depot and each request
depot_loc_ID = 1  
depot_to_requests = {}
for request_id in range(1, len(requests) + 1):
    distance = calculate_depot_to_request_distance(locations, depot_loc_ID, request_id)
    if distance is not None:
        depot_to_requests[request_id] = distance

def distance_between_consecutive_requests(locations, requests, request_id1, request_id2):
    #Calculate the Euclidean distance between the two consecutive requests.

    location_id1 = None
    location_id2 = None

    for request in requests:
        if request[0] == request_id1:
            location_id1 = request[1]
        elif request[0] == request_id2:
            location_id2 = request[1]

    if location_id1 is not None and location_id2 is not None:
        coordinates1 = get_location_coordinates(location_id1, locations)
        coordinates2 = get_location_coordinates(location_id2, locations)
        distance = calculate_distance(coordinates1, coordinates2)
        return distance
    else:
        return None

def calculate_consecutive_request_distances(locations, requests):
    #Calculate and store the distances between all pairs of consecutive requests. Returns dictionary storing the distances between consecutive requests.

    consecutive_distances = {}

    for i in range(len(requests) - 1):
        request_id1 = requests[i][0]
        request_id2 = requests[i + 1][0]
        distance = distance_between_consecutive_requests(locations, requests, request_id1, request_id2)
        if distance is not None:
            consecutive_distances[(request_id1, request_id2)] = distance

    return consecutive_distances

consecutive_distances = calculate_consecutive_request_distances(locations, requests)

def calculate_all_combined_distances(locations, requests, depot_loc_ID):
    #Calc the round trip distances. (depot - first request - second request - depot)

    combined_distances = {}

    for i in range(len(requests)):
        request_id1 = requests[i][0]
        for j in range(i + 1, len(requests)):
            request_id2 = requests[j][0]

            # Calculate the combined distances for the current pair of requests
            depot_to_first_request_distance = calculate_depot_to_request_distance(locations, depot_loc_ID, request_id1)
            consecutive_requests_distance = distance_between_consecutive_requests(locations, requests, request_id1, request_id2)
            second_request_to_depot_distance = calculate_depot_to_request_distance(locations, depot_loc_ID, request_id2)

            # Store the combined distances in the dictionary
            combined_distances[(request_id1, request_id2)] = {
                'depot_to_first_request_distance': depot_to_first_request_distance,
                'consecutive_requests_distance': consecutive_requests_distance,
                'second_request_to_depot_distance': second_request_to_depot_distance
            }

    return combined_distances

total_distances_for_pairs = {}
combined_distances = calculate_all_combined_distances(locations, requests, depot_loc_ID)

for pair, distances in combined_distances.items():
    total_distance_for_pair = (
        distances['depot_to_first_request_distance']
        + distances['consecutive_requests_distance']
        + distances['second_request_to_depot_distance']
    )
    total_distances_for_pairs[pair] = total_distance_for_pair

In [6]:
def check_feasibility_for_all_pairs(total_distances_for_pairs, TRUCK_MAX_DISTANCE):
    #Check the feasibility of total distances for all pairs. Returns dictionary where keys are pairs and values are boolean indicating feasibility.
    feasibility_dict = {}
    for pair, total_distance in total_distances_for_pairs.items():
        feasibility_dict[pair] = total_distance <= TRUCK_MAX_DISTANCE
    return feasibility_dict

feasibility_dict = check_feasibility_for_all_pairs(total_distances_for_pairs, TRUCK_MAX_DISTANCE)

def get_truck_routes_with_days(locations, depot, TRUCK_CAPACITY, requests, machines):
    # Sort requests based on the earliest delivery day
    sorted_requests = sorted(requests, key=lambda req: req[2])
    
    depot_loc_ID = depot[0]
    truck_routes = []
    current_truck_route = []
    current_truck_weight = 0
    current_delivery_day = None
    
    for request_id, location_id, start_day, end_day, machine_kind_id, num_requested_machines in sorted_requests:
        machine_size = [machine[1] for machine in machines if machine[0] == machine_kind_id][0]
        request_weight = machine_size * num_requested_machines
        
        if not current_truck_route:
            current_truck_route.append([request_id])
            current_truck_weight = request_weight
            current_delivery_day = start_day
        elif current_truck_weight + request_weight <= TRUCK_CAPACITY and start_day == current_delivery_day:
            # Check if adding the current request exceeds the truck capacity and if delivery day matches
            combined_request_ids = current_truck_route[-1] + [request_id]
            combined_weight = current_truck_weight + request_weight
            if combined_weight <= TRUCK_CAPACITY:
                current_truck_route[-1] = combined_request_ids
                current_truck_weight = combined_weight
            else:
                truck_routes.append((current_truck_route, current_delivery_day))
                current_truck_route = [[request_id]]
                current_truck_weight = request_weight
                current_delivery_day = start_day
        else:
            # If adding the current request exceeds the truck capacity or delivery day does not match,
            # start a new truck route with the current request
            truck_routes.append((current_truck_route, current_delivery_day))
            current_truck_route = [[request_id]]
            current_truck_weight = request_weight
            current_delivery_day = start_day
    
    # Append the last truck route if it's not empty
    if current_truck_route:
        truck_routes.append((current_truck_route, current_delivery_day))
    
    return truck_routes


def calculate_delivery_days(truck_routes):
    delivery_days = {}
    for truck_route, delivery_day in truck_routes:
        for request_ids in truck_route:
            for request_id in request_ids:
                delivery_days[request_id] = delivery_day
    return delivery_days


truck_routes = get_truck_routes_with_days(locations, depot, TRUCK_CAPACITY, requests, machines)
delivery_days = calculate_delivery_days(truck_routes)

def get_truck_routes_with_constraints(locations, depot, TRUCK_CAPACITY, TRUCK_MAX_DISTANCE, requests, machines, feasibility_dict):
    # Sort requests based on the earliest delivery day
    sorted_requests = sorted(requests, key=lambda req: req[2])
    
    depot_loc_ID = depot[0]
    truck_routes = []
    current_truck_route = []
    current_truck_weight = 0
    current_truck_distance = 0
    current_delivery_day = None
    
    for request_id, location_id, start_day, end_day, machine_kind_id, num_requested_machines in sorted_requests:
        machine_size = [machine[1] for machine in machines if machine[0] == machine_kind_id][0]
        request_weight = machine_size * num_requested_machines
        
        # Calculate the distance from the depot to the current location
        depot_to_location_distance = calculate_request_distance(requests, locations, depot_loc_ID, location_id)
        
        # Check if the distance calculation was successful
        if depot_to_location_distance is None:
            print(f"Error: Unable to calculate distance between depot and request {request_id}")
            continue
        
        # Check if adding the current request meets all constraints
        if (current_truck_weight + request_weight <= TRUCK_CAPACITY and 
            current_truck_distance + depot_to_location_distance <= TRUCK_MAX_DISTANCE and
            start_day == current_delivery_day):
            # If constraints are satisfied, add the request to the current truck route
            if current_truck_route:
                combined_request_ids = current_truck_route[-1] + [request_id]
                if all(feasibility_dict[(req_id1, req_id2)] for req_id1 in current_truck_route[-1] for req_id2 in [request_id]):
                    current_truck_route[-1] = combined_request_ids
                    current_truck_weight += request_weight
                    current_truck_distance += depot_to_location_distance
                else:
                    truck_routes.append((current_truck_route, current_delivery_day))
                    current_truck_route = [[request_id]] 
                    current_truck_weight = request_weight
                    current_truck_distance = depot_to_location_distance
                    current_delivery_day = start_day
            else:
                current_truck_route.append([request_id]) 
                current_truck_weight += request_weight
                current_truck_distance += depot_to_location_distance
        else:
            # If constraints are not satisfied, start a new truck route with the current request
            if current_truck_route:
                truck_routes.append((current_truck_route, current_delivery_day)) 
            current_truck_route = [[request_id]]  
            current_truck_weight = request_weight
            current_truck_distance = depot_to_location_distance
            current_delivery_day = start_day
    

    if current_truck_route:
        truck_routes.append((current_truck_route, current_delivery_day))
    
    return truck_routes

# Calculate truck routes with constraints
truck_routes_with_constraints = get_truck_routes_with_constraints(
    locations, depot, TRUCK_CAPACITY, TRUCK_MAX_DISTANCE, requests, machines, feasibility_dict
)

In [7]:
def constrained_truck_routes_with_depot(truck_routes_with_constraints, depot_loc_ID):
    truck_routes_with_depot = []
    for truck_route, delivery_day in truck_routes_with_constraints:
        # Add depot at the beginning and end of the truck route
        truck_route_with_depot = [depot_loc_ID] + truck_route + [depot_loc_ID]
        truck_routes_with_depot.append((truck_route_with_depot, delivery_day))
    return truck_routes_with_depot


truck_routes_with_constraints = get_truck_routes_with_constraints(locations, depot, TRUCK_CAPACITY, TRUCK_MAX_DISTANCE, requests, machines, feasibility_dict)
truck_routes_with_depot = constrained_truck_routes_with_depot(truck_routes_with_constraints, depot_loc_ID)

def extract_truck_routes(truck_routes_with_depot):
    extracted_routes = []
    for route, delivery_day in truck_routes_with_depot:
        request_ids = []
        for segment in route:
            if isinstance(segment, list): 
                request_ids.extend(segment)
        extracted_routes.append((request_ids, delivery_day))
    return extracted_routes

truck_routes_extracted = extract_truck_routes(truck_routes_with_depot)

def convert_truck_routes_to_locations(truck_routes_with_depot, requests):
    truck_routes_locations = []
    for route, delivery_day in truck_routes_with_depot:
        location_route = [1]  
        for segment in route:
            if isinstance(segment, list):  
                for request_id in segment:
                    location_id = requests[request_id - 1][1]  # Location ID of the request
                    location_route.append(location_id)
        location_route.append(1)  
        truck_routes_locations.append((location_route, delivery_day))
    return truck_routes_locations

truck_routes_locations = convert_truck_routes_to_locations(truck_routes_with_depot, requests)

def calculate_truck_distances(locations, truck_routes_locations):
    truck_distances_per_day = {}
    total_truck_distance = 0
    
    for idx, (route, _) in enumerate(truck_routes_locations, start=1):
        total_distance = 0
        for i in range(len(route) - 1):
            loc1 = locations[route[i] - 1]
            loc2 = locations[route[i + 1] - 1]
            distance = ((loc1[1] - loc2[1]) ** 2 + (loc1[2] - loc2[2]) ** 2) ** 0.5  
            total_distance += distance
        total_distance = round(total_distance, 2) 
        truck_distances_per_day[idx] = total_distance
        total_truck_distance += total_distance
    
    total_truck_distance = ceil(total_truck_distance)
    return truck_distances_per_day, total_truck_distance

truck_distances_per_day, total_truck_distance = calculate_truck_distances(locations, truck_routes_locations)

def calculate_idle_days(requests, delivery_days):
# Calculate the idle days for each machine based on delivery and installation days. Returns dictionary where keys are machine kind IDs and values are lists of idle days.

    idle_days = {}
    for request_id, installation_day in delivery_days.items():
        delivery_day = requests[request_id - 1][2]
        machine_kind_id = requests[request_id - 1][4] 
        idle_days[machine_kind_id] = max(0, installation_day - delivery_day)
    return idle_days

idle_days = calculate_idle_days(requests, delivery_days)

def calculate_penalties(delivery_days, requests):
    penalties = {}
    for request_id, installation_day in delivery_days.items():
        delivery_day = requests[request_id - 1][2]
        machine_kind_id = requests[request_id - 1][4]
        idle_days = max(0, installation_day - delivery_day - 1) 
        if machine_kind_id not in penalties:
            penalties[machine_kind_id] = 0
        penalties[machine_kind_id] += idle_days
    return penalties



In [8]:
#Dit is het eind van de truck routes, hier start het Technician-gedeelte



In [9]:
def get_location_for_request(request):
    return requests[request - 1][1]

def nearest_neighbor(point, unvisited_requests, locations):
    min_distance = float('inf')
    nearest_point = None
    point_coordinates = get_location_coordinates(point, locations)
    
    for candidate_request in unvisited_requests:
        request_location_id = get_location_for_request(candidate_request)
        if request_location_id != point:
            candidate_coordinates = get_location_coordinates(request_location_id, locations)
            dist = calculate_distance(point_coordinates, candidate_coordinates)
            if dist is not None and dist < min_distance:
                min_distance = dist
                nearest_point = request_location_id
    return nearest_point

def get_requests_for_location(requests, location_id):
    return [req[0] for req in requests if req[1] == location_id]

for location in locations:
    location_id = location[0]
    location_request = [get_requests_for_location(requests, location_id)]

technician_details = {}

def initialize_technician_details(technicians):
    global technician_details
    technician_details.clear()
    for technician in technicians:
        tech_id = technician[0]
        max_distance = technician[2]
        max_installations = technician[3]
        technician_details[tech_id] = {'max_distance': max_distance, 'max_installations': max_installations}

initialize_technician_details(technicians)

max_distance = {tech_id: details['max_distance'] for tech_id, details in technician_details.items()}
max_installations = {tech_id: details['max_installations'] for tech_id, details in technician_details.items()}

def initialize_home_locations(technicians):
    global home_location_technician
    home_location_technician = {tech[0]: tech[1] for tech in technicians}
    return home_location_technician

initialize_home_locations(technicians)

def nearest_neighbor_for_technician_home(technicians, unvisited_requests, locations):
    home_location_technician = initialize_home_locations(technicians)
    
    nearest_locations = {}
    for technician_id, home_location in home_location_technician.items():
        nearest_location = nearest_neighbor(home_location, unvisited_requests, locations)
        nearest_locations[technician_id] = nearest_location
    
    return nearest_locations

def transform_skill_set(technicians, technician_id):
    for technician in technicians:
        if technician[0] == technician_id:
            skill_set = technician[4:]
            transformed_skills = []
            for index, value in enumerate(skill_set):
                if value != 0:
                    transformed_skills.append(index + 1)
                else:
                    transformed_skills.append(0)
            return transformed_skills
    return None

def doable_request_skillbased(technicians, requests):
    doable_requests = {}
    for technician in technicians:
        technician_id = technician[0]
        skill_set = transform_skill_set(technicians, technician_id)
        for request in requests:
            request_id = request[0]
            machine_type = request[4]
            if machine_type in skill_set:
                if technician_id not in doable_requests:
                    doable_requests[technician_id] = []
                doable_requests[technician_id].append(request_id)
    return doable_requests

doable_requests = doable_request_skillbased(technicians, requests)
doable_requests_copy = doable_request_skillbased(technicians, requests)  # Ensuring this variable is created

def check_location_requests_doable(technicians, requests, locations):
    location_results = {}
    for location in locations:
        location_id = location[0]
        location_requests = [request for request in requests if request[1] == location_id]
        location_doable_requests = {}
        for technician in technicians:
            technician_id = technician[0]
            skill_set = transform_skill_set(technicians, technician_id)
            doable_requests = [request[0] for request in location_requests if request[4] in skill_set]
            if doable_requests:
                location_doable_requests[technician_id] = doable_requests
        location_results[location_id] = location_doable_requests
    return location_results

location_results = check_location_requests_doable(technicians, requests, locations)

unvisited_requests = [1, 2, 3, 4, 5]
current_location = 2

nearest = nearest_neighbor(current_location, unvisited_requests, locations)
nearest_locations = nearest_neighbor_for_technician_home(technicians, unvisited_requests, locations)

def rare_requests(doable_requests_copy):
    #Identify the requests that can be handled by the fewest number of technicians.
    request_count = {}
    for tech_id, requests in doable_requests_copy.items():
        for request_id in requests:
            if request_id in request_count:
                request_count[request_id] += 1
            else:
                request_count[request_id] = 1

    if not request_count:
        return {}

    min_count = min(request_count.values())
    rare_requests_dict = {req_id: count for req_id, count in request_count.items() if count == min_count}

    return rare_requests_dict


rare_requests_dict = rare_requests(doable_requests_copy)


In [10]:
def extract_machine_count(requests):
    machine_count = {}
    for req in requests:
        machine_count[req[0]] = req[5]
    return machine_count
machine_count = extract_machine_count(requests)

def extract_delivery_schedule(truck_routes):
    delivery_schedule = {}
    for route in truck_routes:
        requests, delivery_day = route
        for req in requests:
            for request_id in req:
                delivery_schedule[request_id] = delivery_day
    return delivery_schedule

delivery_schedule = extract_delivery_schedule(truck_routes)
num_requests = len(delivery_schedule)

def extract_assignment_days(truck_routes):
    assignment_schedule = {}
    for route in truck_routes:
        requests, delivery_day = route
        for req in requests:
            for request_id in req:
                assignment_schedule[request_id] = delivery_day + 1
    return assignment_schedule

assignment_schedule = extract_assignment_days(truck_routes)


In [11]:
def doable_request_skillbased(technicians, requests):
    doable_requests = {} 
    for technician in technicians:
        technician_id = technician[0]
        skill_set = transform_skill_set(technicians, technician_id)
        
        for request in requests:
            request_id = request[0]
            machine_type = request[4] 

            if machine_type in skill_set:
                if technician_id not in doable_requests:
                    doable_requests[technician_id] = []
                doable_requests[technician_id].append(request_id)
    
    return doable_requests

doable_requests = doable_request_skillbased(technicians, requests)

for technician_id, request_ids in doable_requests.items():
    transformed_skills = transform_skill_set(technicians, technician_id)
    

def home_location_technician_coordinates(home_location_technician, locations):
    #Get the coordinates of technicians' home locations.

    home_coordinates = {}
    for tech_id, location_id in home_location_technician.items():
        coordinates = get_location_coordinates(location_id, locations)
        if coordinates:
            home_coordinates[tech_id] = coordinates
    return home_coordinates

home_coordinates = home_location_technician_coordinates(home_location_technician, locations)

def request_location_coordinates(requests, locations):
    #Get the coordinates of each request location.

    request_coordinates = {}
    for request in requests:
        request_id, location_id, *_ = request
        coordinates = get_location_coordinates(location_id, locations)
        if coordinates:
            request_coordinates[request_id] = coordinates
    return request_coordinates

request_coordinates = request_location_coordinates(requests, locations)
    
def get_request_location_ids(requests):
    #Extract and print the location IDs of each request.
    location_ids = [request[1] for request in requests]
    return location_ids

location_ids = get_request_location_ids(requests)

def distance_between_home_and_request(home_coordinates, request_coordinates, home_id, request_id):
    #Calculate the distance between a home location and a request location.
    home_location = home_coordinates.get(home_id)
    request_location = request_coordinates.get(request_id)
    if home_location is None:
        raise ValueError(f"Home ID {home_id} not found in home_coordinates.")
    if request_location is None:
        raise ValueError(f"Request ID {request_id} not found in request_coordinates.")
    
    return calculate_distance(home_location, request_location)

def calculate_distances_for_all_technicians_and_requests(home_coordinates, request_coordinates):
    #Calculate and store distances between each technician's home and each request. Returns dictionary with keys as (home_id, request_id) tuples and values as distances.

    distances = {}
    for home_id, home_location in home_coordinates.items():
        for request_id, request_location in request_coordinates.items():
            distance = distance_between_home_and_request(home_coordinates, request_coordinates, home_id, request_id)
            distances[(home_id, request_id)] = distance
    return distances


distances = calculate_distances_for_all_technicians_and_requests(home_coordinates, request_coordinates)
def check_max_distance_exceeded(distances, max_distance):
    #Check if the round trip distance for each technician exceeds their maximum distance.
    results = {}
    for (home_id, request_id), distance in distances.items():
        round_trip_distance = 2 * distance
        max_dist = max_distance.get(home_id, float('inf'))
        exceeds_max_distance = round_trip_distance > max_dist
        results[(home_id, request_id)] = exceeds_max_distance
    return results

exceeded_distances = check_max_distance_exceeded(distances, max_distance)


In [12]:
def doable_requests_distance_based(exceeded_distances):
    #Check which requests each technician can handle based on the distances and maximum distances.
    doable_requests = {}
    for (home_id, request_id), exceeds in exceeded_distances.items():
        if not exceeds:
            if home_id not in doable_requests:
                doable_requests[home_id] = []
            doable_requests[home_id].append(request_id)
    return doable_requests


doable_requests_distance_based = doable_requests_distance_based(exceeded_distances)

def doable_requests_double_check(doable_requests_distance_based, doable_requests):
    #makes a list of requests based on a technicians skillset and dsitances from home
    double_check_requests = {}
    for tech_id in doable_requests_distance_based.keys():
        common_requests = set(doable_requests_distance_based[tech_id]) & set(doable_requests[tech_id])
        double_check_requests[tech_id] = list(common_requests)
    return double_check_requests

double_check_requests = doable_requests_double_check(doable_requests_distance_based, doable_requests)

def rare_requests_double_checked(double_check_requests):
    # Count how many technicians can handle each request and keep track of the technicians
    request_count = {}
    request_technicians = {}
    for tech_id, tech_requests in double_check_requests.items():
        for request in tech_requests:
            request_count[request] = request_count.get(request, 0) + 1
            request_technicians.setdefault(request, []).append(tech_id)
    
    min_count = min(request_count.values()) if request_count else 0
    rare_requests_dict = {req_id: {'count': count, 'technicians': request_technicians[req_id]} for req_id, count in request_count.items() if count == min_count}
    
    return rare_requests_dict

rare_requests_dict = rare_requests_double_checked(double_check_requests)


def technician_rare_request_count(double_check_requests, rare_requests):
    # Initialize a dictionary to store the rare request count for each technician
    technician_rare_counts = {tech_id: 0 for tech_id in double_check_requests.keys()}

    for req_id, data in rare_requests.items():
        technicians = data['technicians']
        for tech_id in technicians:
            technician_rare_counts[tech_id] += 1
    
    return technician_rare_counts

technician_rare_counts = technician_rare_request_count(double_check_requests, rare_requests_dict)

def find_requests_with_same_day_and_loc(assignment_schedule, request_coordinates, location_ids):
    #Find requests with the same location ID and assignment day.

    requests_same_day_loc = {}
    for request_id, assignment_day in assignment_schedule.items():
        location_id = location_ids[request_id - 1]  
        if location_id is not None:
            key = (location_id, assignment_day)
            requests_same_day_loc.setdefault(key, []).append(request_id)
    return {key: requests for key, requests in requests_same_day_loc.items() if len(requests) > 1}

requests_same_day_loc = find_requests_with_same_day_and_loc(assignment_schedule, request_coordinates, location_ids)

def find_request_pairs_across_days(assignment_schedule, request_coordinates):
    #Find pairs of requests and calculate the distance between them.

    request_pairs_across_days = {}
    max_assignment_day = max(assignment_schedule.values())

    for day in range(1, max_assignment_day + 1):
        requests_to_consider = [req for req, assigned_day in assignment_schedule.items() if assigned_day <= day]

        pairs_with_distance = []
        for req1, req2 in combinations(requests_to_consider, 2):
            coord1 = request_coordinates[req1]
            coord2 = request_coordinates[req2]
            distance = calculate_distance(coord1, coord2)
            pairs_with_distance.append(((req1, req2), distance))
        
        request_pairs_across_days[day] = pairs_with_distance

    return request_pairs_across_days

request_pairs_across_days = find_request_pairs_across_days(assignment_schedule, request_coordinates)




In [13]:
def calculate_round_trip_distances(home_coordinates, request_pairs_across_days, request_coordinates):
    #Calculate the round-trip distances for the technician. Home to 1st request to 2nd request and back home.

    technician_round_trip_distances = {}
    for technician, home_coord in home_coordinates.items():
        round_trip_distances = {}
        for day, pairs_distance in request_pairs_across_days.items():
            day_distances = []
            for pair, distance in pairs_distance:
                request_id1, request_id2 = pair
                coord1 = request_coordinates[request_id1]
                coord2 = request_coordinates[request_id2]
                distance_to_first = calculate_distance(home_coord, coord1)
                distance_from_second = calculate_distance(coord2, home_coord)
                round_trip_distance = distance_to_first + distance + distance_from_second
                day_distances.append((pair, round_trip_distance))
            round_trip_distances[day] = day_distances
        technician_round_trip_distances[technician] = round_trip_distances
    return technician_round_trip_distances


request_pairs_across_days = find_request_pairs_across_days(assignment_schedule, request_coordinates)
technician_round_trip_distances = calculate_round_trip_distances(home_coordinates, request_pairs_across_days, request_coordinates)


def find_lowest_distance_pairs(technician_round_trip_distances):
    #Find the two request pairs with the lowest round-trip distances for each technician on each day. 
    
    lowest_distance_pairs = {}
    for technician, round_trip_distances in technician_round_trip_distances.items():
        tech_lowest_pairs = {}
        for day, pairs_distance in round_trip_distances.items():
            sorted_pairs = sorted(pairs_distance, key=lambda x: x[1])[:2]
            tech_lowest_pairs[day] = sorted_pairs
        lowest_distance_pairs[technician] = tech_lowest_pairs
    return lowest_distance_pairs

request_pairs_across_days = find_request_pairs_across_days(assignment_schedule, request_coordinates)
technician_round_trip_distances = calculate_round_trip_distances(home_coordinates, request_pairs_across_days, request_coordinates)

lowest_distance_pairs = find_lowest_distance_pairs(technician_round_trip_distances)



def can_handle_request_combinations(technician_round_trip_distances, max_distance):
    #Determine if a technician can handle the request combinations within their maximum travel distance.

    feasible_combinations = {}
    for technician, round_trip_distances in technician_round_trip_distances.items():
        tech_feasible_pairs = {}
        for day, pairs_distance in round_trip_distances.items():
            feasible_pairs = [
                (pair, distance) for pair, distance in pairs_distance if distance <= max_distance.get(technician, float('inf'))
            ]
            tech_feasible_pairs[day] = feasible_pairs
        feasible_combinations[technician] = tech_feasible_pairs
    return feasible_combinations


request_pairs_across_days = find_request_pairs_across_days(assignment_schedule, request_coordinates)
technician_round_trip_distances = calculate_round_trip_distances(home_coordinates, request_pairs_across_days, request_coordinates)
lowest_distance_pairs = find_lowest_distance_pairs(technician_round_trip_distances)
feasible_combinations = can_handle_request_combinations(technician_round_trip_distances, max_distance)



def filter_combinations_by_skill_set(feasible_combinations, doable_requests_copy):
    #Filter the distance-based feasible request combinations for the technician's skill set.

    skill_filtered_combinations = {}
    for technician, day_pairs in feasible_combinations.items():
        tech_skill_filtered = {}
        for day, pairs_distance in day_pairs.items():
            filtered_pairs = [
                (pair, distance) for pair, distance in pairs_distance
                if pair[0] in doable_requests_copy.get(technician, []) and pair[1] in doable_requests_copy.get(technician, [])
            ]
            tech_skill_filtered[day] = filtered_pairs
        skill_filtered_combinations[technician] = tech_skill_filtered
    return skill_filtered_combinations

feasible_combinations = can_handle_request_combinations(technician_round_trip_distances, max_distance)
skill_filtered_combinations = filter_combinations_by_skill_set(feasible_combinations, doable_requests_copy)

def filter_combinations_by_max_installations(skill_filtered_combinations, machine_count, max_installations):
    #Filter the skill filtered combinations to not exceed the technician's maximum installation capacity.

    final_filtered_combinations = {}
    for technician, day_pairs in skill_filtered_combinations.items():
        tech_filtered = {}
        for day, pairs_distance in day_pairs.items():
            filtered_pairs = [
                (pair, distance) for pair, distance in pairs_distance
                if machine_count[pair[0]] + machine_count[pair[1]] <= max_installations[technician]
            ]
            tech_filtered[day] = filtered_pairs
        final_filtered_combinations[technician] = tech_filtered
    return final_filtered_combinations

final_filtered_combinations = filter_combinations_by_max_installations(skill_filtered_combinations, machine_count, max_installations)


In [14]:
def check_tech_req_same_day(double_check_requests, request_coordinates, assignment_schedule, location_ids):
    #Check if technicians can do requests with the same location and assignment day. Returns dictionary where keys are tuples (technician ID, assignment day) and values are lists of request IDs.
    tech_req_same_day = {}
    for tech_id, requests in double_check_requests.items():
        req_same_day = {}

        for req_id in requests:
            location_id = location_ids[req_id - 1]  
            day = assignment_schedule[req_id]
            key = (location_id, day)
            req_same_day.setdefault(key, []).append(req_id)
        req_same_day = {key: req_ids for key, req_ids in req_same_day.items() if len(req_ids) > 1}
        if req_same_day:
            tech_req_same_day[tech_id] = req_same_day

    return tech_req_same_day

tech_req_same_day = check_tech_req_same_day(double_check_requests, request_coordinates, assignment_schedule, location_ids)

def possible_tech_combis_on_day(tech_req_same_day):
    #Create pairs of requests for technicians who can handle requests with the same location ID and assignment day. Returns List of tuples where each tuple contains a pair of requests for a technician.
    tech_combis = []

    for tech_id, req_same_day in tech_req_same_day.items():
        for req_ids in req_same_day.values():
            for i in range(len(req_ids) - 1):
                tech_combis.append((tech_id, req_ids[i], req_ids[i + 1]))

    return tech_combis

tech_combis = possible_tech_combis_on_day(tech_req_same_day)

def weighted_tech_combis(tech_combis, machine_count):
    #Calc the weight of each tech combi based on the machine count of the requests it contains.

    weighted_combis = {}
    
    for tech_id, req1, req2 in tech_combis:
        weight = machine_count.get(req1, 0) + machine_count.get(req2, 0)
        weighted_combis[(tech_id, req1, req2)] = weight
    
    return weighted_combis

weighted_combis = weighted_tech_combis(tech_combis, machine_count)

def feasible_tech_combi(weighted_combis, max_installations):
    #Check if each tech combi is feasible for a technician based on their maximum installations per day. Returns dctionary with tech combis as keys and boolean values indicating feasibility.

    feasible_combis = {}
    for tech_combi, combi_weight in weighted_combis.items():
        tech_id = tech_combi[0]
        max_installation = max_installations.get(tech_id, 0)
        feasible_combis[tech_combi] = combi_weight <= max_installation
    return feasible_combis

feasible_combis = feasible_tech_combi(weighted_combis, max_installations)

final_filtered_combinations_dict = {}

for tech_id, days_combinations in final_filtered_combinations.items():
    for day, pairs_distances in days_combinations.items():
        for pair_distance in pairs_distances:
            if pair_distance:
                req1, req2 = pair_distance[0]
                final_filtered_combinations_dict[(tech_id, req1, req2)] = True

def add_third_request_to_pairs(final_filtered_combinations_dict, location_ids, machine_count, max_installations):
    triple_combinations_dict = {}
    
    for (tech_id, req1, req2) in final_filtered_combinations_dict.keys():
        location1 = location_ids[req1 - 1]
        location2 = location_ids[req2 - 1]
        
        for req3, loc3 in enumerate(location_ids, start=1):
            if req3 in (req1, req2):
                continue
            
            if machine_count[req1] + machine_count[req2] + machine_count[req3] > max_installations[tech_id]:
                continue
            
            if loc3 == location1:
                triple_combinations_dict[(tech_id, req1, req3, req2)] = True
            elif loc3 == location2:
                triple_combinations_dict[(tech_id, req1, req2, req3)] = True

    return triple_combinations_dict

triple_combinations_dict = add_third_request_to_pairs(final_filtered_combinations_dict, location_ids, machine_count, max_installations)



In [15]:
def time_window_for_requests(requests):
    time_windows = {}
    for request_info in requests:
        request_id = request_info[0]
        start_day = request_info[2]
        end_day = request_info[3]
        duration = end_day - start_day + 1
        time_windows[request_id] = duration
    return time_windows


time_windows = time_window_for_requests(requests)

def technician_rare_request_count(double_check_requests, rare_requests):
    technician_rare_counts = {tech_id: 0 for tech_id in double_check_requests.keys()}
    
    for req_id, data in rare_requests.items():
        technicians = data['technicians']
        for tech_id in technicians:
            technician_rare_counts[tech_id] += 1
    
    return technician_rare_counts

In [16]:

technician_rare_requests = {tech_id: 0 for tech_id in double_check_requests.keys()}

def update_available_technicians(current_day, available_technicians, technician_rest_days, consecutive_days, last_day_worked, technician_rest_days_details):
    for tech_id in list(available_technicians):
        rest_days_remaining = technician_rest_days.get(tech_id, 0)
        consecutive_days_worked = consecutive_days.get(tech_id, 0)
        last_day = last_day_worked.get(tech_id, None)
        rest_day_1, rest_day_2 = technician_rest_days_details.get(tech_id, (None, None))

        # Calculate rest_days_remaining based on current day and rest days
        if rest_day_1 is not None and current_day == rest_day_1:
            rest_days_remaining = 2
        elif rest_day_2 is not None and current_day == rest_day_2:
            rest_days_remaining = 1
        elif rest_day_2 is not None and current_day > rest_day_2:
            rest_days_remaining = 0

        technician_rest_days[tech_id] = rest_days_remaining

        if (rest_day_1 is not None and rest_day_1 == current_day) or (rest_day_2 is not None and rest_day_2 == current_day):
            if tech_id in available_technicians:
                available_technicians.remove(tech_id)
            continue

        if rest_days_remaining > 0:
            if current_day in (rest_day_1, rest_day_2):
                technician_rest_days[tech_id] -= 1
                if tech_id in available_technicians:
                    available_technicians.remove(tech_id)
        else:
            if consecutive_days_worked >= 4:
                technician_rest_days[tech_id] = 2
                consecutive_days[tech_id] = 0
                technician_rest_days_details[tech_id] = (current_day, current_day + 1)

                if tech_id in available_technicians:
                    available_technicians.remove(tech_id)


def try_assign_request(request_id, day_to_assign, available_technicians, assigned_requests, assigned_request_ids, consecutive_days, last_day_worked, later_deliveries, final_filtered_combinations_dict, triple_combinations_dict, technician_rare_requests, technician_rest_days, assignment_schedule, double_check_requests):
    available_technicians = {tech_id for tech_id in available_technicians if technician_rest_days.get(tech_id, 0) == 0}

    if not available_technicians:
        return False

    min_rare_count = min(technician_rare_requests.get(tech_id, 0) for tech_id in available_technicians)
    min_consecutive_days = min(consecutive_days.get(tech_id, 0) for tech_id in available_technicians if technician_rare_requests.get(tech_id, 0) == min_rare_count)
    suitable_technicians = [tech_id for tech_id in available_technicians if technician_rare_requests.get(tech_id, 0) == min_rare_count and consecutive_days.get(tech_id, 0) == min_consecutive_days]

    if not suitable_technicians:
        return False

    min_assigned = min(suitable_technicians, key=lambda tech_id: (technician_rare_requests.get(tech_id, 0), consecutive_days.get(tech_id, 0)))

    # Check the number of requests already assigned to the technician on the same day
    requests_assigned_today = [request for request in assigned_requests.get(min_assigned, []) if request[1] == day_to_assign]
    num_requests_assigned_today = len(requests_assigned_today)

  
    if num_requests_assigned_today >= 3:
        return False

    if num_requests_assigned_today == 0:
        if request_id in double_check_requests[min_assigned]:
            assigned_requests.setdefault(min_assigned, []).append((request_id, day_to_assign))
            assigned_request_ids.add(request_id)
        else:

            return False

    elif num_requests_assigned_today == 1:
        existing_request = requests_assigned_today[0][0]
        if (min_assigned, min(existing_request, request_id), max(existing_request, request_id)) in final_filtered_combinations_dict:
            assigned_requests[min_assigned].append((request_id, day_to_assign))
            assigned_request_ids.add(request_id)

        else:

            return False

    elif num_requests_assigned_today == 2:
        existing_requests = [request[0] for request in requests_assigned_today]
        if (min_assigned, *sorted(existing_requests + [request_id])) in triple_combinations_dict:
            assigned_requests[min_assigned].append((request_id, day_to_assign))
            assigned_request_ids.add(request_id)

        else:

            return False

    # Update consecutive days and last day worked
    if last_day_worked.get(min_assigned) is not None:
        if day_to_assign == last_day_worked[min_assigned] + 1:
            consecutive_days[min_assigned] += 1
        elif day_to_assign != last_day_worked[min_assigned]:
            consecutive_days[min_assigned] = 1  # Reset if non-consecutive day
    last_day_worked[min_assigned] = day_to_assign

    
    # Check for later deliveries
    if day_to_assign > assignment_schedule[request_id]:
        later_deliveries[request_id] = day_to_assign - assignment_schedule[request_id]

    return True

def assign_requests(assignment_schedule, double_check_requests, days, technician_rare_requests, final_filtered_combinations_dict, triple_combinations_dict):
    assigned_requests = {}
    assigned_request_ids = set()
    unassigned_requests = set()
    later_deliveries = {}
    consecutive_days = {tech_id: 0 for tech_id in double_check_requests.keys()}
    last_day_worked = {tech_id: None for tech_id in double_check_requests.keys()}
    technician_rest_days = {tech_id: 0 for tech_id in double_check_requests.keys()}
    technician_rest_days_details = {tech_id: (None, None) for tech_id in double_check_requests.keys()}


    for request_id, assignment_day in sorted(assignment_schedule.items(), key=lambda x: x[1]):
        if request_id in assigned_request_ids:
            continue

        assigned = False
        for day_delay in range(days + 1):
            day_to_assign = assignment_day + day_delay
            if day_to_assign > days:
                unassigned_requests.add(request_id)
                break

            available_technicians = {tech_id for tech_id, requests in double_check_requests.items() if request_id in requests}
            update_available_technicians(day_to_assign, available_technicians, technician_rest_days, consecutive_days, last_day_worked, technician_rest_days_details)

            if not available_technicians:
                continue

            if try_assign_request(request_id, day_to_assign, available_technicians, assigned_requests, assigned_request_ids, consecutive_days, last_day_worked, later_deliveries, final_filtered_combinations_dict, triple_combinations_dict, technician_rare_requests, technician_rest_days, assignment_schedule, double_check_requests):
                assigned = True
                break

        if not assigned:
            unassigned_requests.add(request_id)

    return assigned_requests, unassigned_requests, later_deliveries

technician_rare_requests = {tech_id: 0 for tech_id in double_check_requests.keys()}
assigned_requests, unassigned_requests, later_deliveries = assign_requests(assignment_schedule, double_check_requests, DAYS, technician_rare_requests, final_filtered_combinations_dict, triple_combinations_dict)


In [17]:
final_filtered_combinations_dict = {}

# Transform final_filtered_combinations into the desired format
for tech_id, days_combinations in final_filtered_combinations.items():
    for day, pairs_distances in days_combinations.items():
        for pair_distance in pairs_distances:
            if pair_distance:
                req1, req2 = pair_distance[0]
                final_filtered_combinations_dict[(tech_id, req1, req2)] = True

def idle_request_days(later_deliveries):
    # Calculate the number of days each request is assigned later than its scheduled day. Returns dictionary where keys are request IDs and values are the number of days each request is late.
    idle_days = {}
    for request_id, delay in later_deliveries.items():
        idle_days[request_id] = delay

    return idle_days

idle_days = idle_request_days(later_deliveries)

# Print idle request days
print("\nIdle request days:")
for request_id, days_late in idle_days.items():
    print(f"Request {request_id} is {days_late} day(s) too late")

def get_idle_days_per_machine(idle_days, requests):
    # Calculate the total idle days for each machine type. Returns dictionary where keys are machine types and values are the total number of idle days.
    machine_types = get_machine_type_from_request(requests)
    machine_numbers = get_machine_count_from_request(requests)
    machine_idle_days = {}

    for request_id, days_late in idle_days.items():
        machine_type = machine_types.get(request_id)
        machine_number = machine_numbers.get(request_id)
        if machine_type:
            if machine_type in machine_idle_days:
                machine_idle_days[machine_type] += days_late * machine_number  # Multiply by machine number here
            else:
                machine_idle_days[machine_type] = days_late * machine_number  # Multiply by machine number here
    return machine_idle_days

machine_idle_days = get_idle_days_per_machine(idle_days, requests)
print("Total Idle Days for Each Machine Type:")
for machine_type, total_idle_days in machine_idle_days.items():
    print(f"{machine_type}: {total_idle_days} day(s)")

def calculate_idle_costs(machine_idle_days, penalty_costs_machines):
    # Calculate the costs of idle machine days.
    idle_costs = {}
    for machine_type, idle_days in machine_idle_days.items():
        penalty_cost = penalty_costs_machines.get(machine_type, 0)
        idle_costs[machine_type] = idle_days * penalty_cost
    return idle_costs

idle_costs = calculate_idle_costs(machine_idle_days, penalty_costs_machines)

print("Total Costs of Idle Days for Each Machine Type:")
total_idle_costs = 0
for machine_type, total_costs in idle_costs.items():
    print(f"{machine_type}: {total_costs} cost units")
    total_idle_costs += total_costs

print(f"Total Idle Costs: {total_idle_costs} cost units")



Idle request days:
Request 3 is 1 day(s) too late
Request 7 is 1 day(s) too late
Total Idle Days for Each Machine Type:
2: 1 day(s)
3: 2 day(s)
Total Costs of Idle Days for Each Machine Type:
2: 571 cost units
3: 1462 cost units
Total Idle Costs: 2033 cost units


In [18]:
def format_assigned_requests(assigned_requests, current_day=None):
    formatted_output = ""
    requests_by_day = {}
    for technician, requests in assigned_requests.items():
        for request, day in requests:
            requests_by_day.setdefault(day, []).append((technician, request))
    
    if current_day is not None:
        sorted_days = [current_day]
    else:
        sorted_days = sorted(requests_by_day.keys())

    for day in sorted_days:
        formatted_output += f"DAY {day}\n"
        requests_on_day = requests_by_day.get(day, [])
        num_technicians = len(set(technician for technician, _ in requests_on_day))
        formatted_output += f"NUMBER_OF_TECHNICIANS = {num_technicians}\n"
        requests_by_technician = {}
        for technician, request in requests_on_day:
            requests_by_technician.setdefault(technician, []).append(request)
        
        for technician, requests in requests_by_technician.items():
            formatted_output += f"{technician} {' '.join(str(request) for request in requests)}\n"
        
        formatted_output += "\n"  
    
    return formatted_output

def store_technician_routes(assigned_requests):
    technician_routes = defaultdict(lambda: defaultdict(list))
    
    # Group requests by technician and day
    for technician, requests in assigned_requests.items():
        for request, day in requests:
            technician_routes[technician][day].append(request)
    
    return technician_routes

technician_routes = store_technician_routes(assigned_requests)

def store_all_technician_routes(technician_routes):
    all_tech_routes = []
    for technician, days in technician_routes.items():
        for day, requests in days.items():
            all_tech_routes.extend([(technician, request, day) for request in requests])
    return all_tech_routes

all_tech_routes = store_all_technician_routes(technician_routes)

def calculate_distances_for_all_technician_routes(technician_routes, home_coordinates, request_coordinates, distances, feasible_combinations):
    tech_route_distances_per_day = defaultdict(lambda: defaultdict(int))
    unique_technicians_used = set()  # Track unique technicians used

    for technician, days in technician_routes.items():
        home_coord = home_coordinates[technician]
        for day, requests in days.items():
            if len(requests) == 1:
                req1 = requests[0]
                coord1 = request_coordinates[req1]
                distance_to_first = calculate_distance(home_coord, coord1)
                distance_from_first = calculate_distance(coord1, home_coord)
                round_trip_distance = distance_to_first + distance_from_first
                tech_route_distances_per_day[technician][day] += round_trip_distance
            elif len(requests) == 2:
                req1, req2 = requests
                coord1 = request_coordinates[req1]
                coord2 = request_coordinates[req2]
                distance_to_first = calculate_distance(home_coord, coord1)
                distance_between = calculate_distance(coord1, coord2)
                distance_from_second = calculate_distance(coord2, home_coord)
                round_trip_distance = distance_to_first + distance_between + distance_from_second
                tech_route_distances_per_day[technician][day] += round_trip_distance
            elif len(requests) == 3:
                req1, req2, req3 = requests
                coord1 = request_coordinates[req1]
                coord2 = request_coordinates[req2]
                coord3 = request_coordinates[req3]
                distance_to_first = calculate_distance(home_coord, coord1)
                distance_between_1_and_2 = calculate_distance(coord1, coord2)
                distance_between_2_and_3 = calculate_distance(coord2, coord3)
                distance_from_third = calculate_distance(coord3, home_coord)
                round_trip_distance = distance_to_first + distance_between_1_and_2 + distance_between_2_and_3 + distance_from_third
                tech_route_distances_per_day[technician][day] += round_trip_distance

            unique_technicians_used.add(technician)  # Add technician to the set

    return tech_route_distances_per_day, unique_technicians_used

tech_route_distances_per_day, unique_technicians_used = calculate_distances_for_all_technician_routes(
    technician_routes, home_coordinates, request_coordinates, distances, feasible_combinations
)

print("Technician route distances per day:")
for technician, distances_per_day in tech_route_distances_per_day.items():
    for day, distance in distances_per_day.items():
        print(f"Technician {technician} on day {day} total distance: {distance}")

def calculate_total_distance_for_all_technicians(tech_route_distances_per_day):
    total_distance = sum(sum(day_distances.values()) for day_distances in tech_route_distances_per_day.values())
    return total_distance

total_distance_all_techs = calculate_total_distance_for_all_technicians(tech_route_distances_per_day)
print(total_distance_all_techs)

Technician route distances per day:
Technician 8 on day 2 total distance: 971
Technician 8 on day 3 total distance: 0
Technician 8 on day 6 total distance: 908
Technician 8 on day 7 total distance: 566
Technician 1 on day 2 total distance: 784
Technician 1 on day 3 total distance: 870
Technician 1 on day 8 total distance: 58
Technician 1 on day 9 total distance: 58
Technician 6 on day 2 total distance: 430
Technician 6 on day 3 total distance: 430
Technician 6 on day 11 total distance: 565
Technician 6 on day 12 total distance: 430
Technician 3 on day 2 total distance: 250
Technician 3 on day 3 total distance: 250
Technician 3 on day 9 total distance: 250
Technician 3 on day 10 total distance: 266
Technician 4 on day 3 total distance: 0
Technician 4 on day 6 total distance: 0
Technician 4 on day 10 total distance: 0
Technician 2 on day 4 total distance: 0
Technician 2 on day 5 total distance: 0
Technician 2 on day 10 total distance: 24
Technician 2 on day 11 total distance: 24
Technici

In [19]:
def print_day_schedule(current_day, truck_routes, assigned_requests):
    print()  # Print an empty line before each day
    print(f"DAY = {current_day}")
    
    truck_routes_for_day = [route for route, day in truck_routes if day == current_day]
    num_trucks_for_day = len(truck_routes_for_day)
    print(f"NUMBER_OF_TRUCKS = {num_trucks_for_day}")
    for truck_index, route in enumerate(truck_routes_for_day, start=1):
        print(truck_index, end=' ')
        for request_id in route:
            print(request_id, end=' ') 
        print() 

    formatted_output = format_assigned_requests(assigned_requests, current_day)
    formatted_output = '\n'.join(formatted_output.split('\n')[1:])
    print(formatted_output)

def calculate_costs(total_truck_distance, total_distance_all_techs, total_idle_costs, truck_usage_count, number_of_technician_days, max_trucks_used, max_technicians_used):
    total_truck_distance_cost = total_truck_distance * TRUCK_DISTANCE_COST
    total_truck_day_cost = truck_usage_count * TRUCK_DAY_COST
    total_truck_cost = max_trucks_used * TRUCK_COST
    total_technician_distance_cost = total_distance_all_techs * TECHNICIAN_DISTANCE_COST
    total_technician_day_cost = number_of_technician_days * TECHNICIAN_DAY_COST
    total_technician_cost = max_technicians_used * TECHNICIAN_COST

    total_cost = (total_truck_distance_cost + total_truck_day_cost + total_truck_cost +
                  total_technician_distance_cost + total_technician_day_cost + total_technician_cost + total_idle_costs)
    return total_cost

In [20]:
def print_schedule(truck_routes, delivery_days, assigned_requests, total_truck_distance, tech_route_distances_per_day, output_file_name, unique_technicians_used):
    total_distance_all_techs = calculate_total_distance_for_all_technicians(tech_route_distances_per_day)
    
    all_days = set(delivery_days.values())
    for requests in assigned_requests.values():
        for _, day in requests:
            all_days.add(day)

    all_days = sorted(all_days)
    total_days = max(all_days) if all_days else 0

    max_trucks_used = 0
    max_technicians_used = 0
    truck_usage_count = 0
    technician_usage_count = 0
    number_of_truck_days = 0
    number_of_technician_days = 0

    daily_output = []

    for current_day in range(1, total_days + 1):
        num_trucks_for_day = 0
        num_technicians_for_day = 0

        day_output = []
        day_output.append(f"DAY = {current_day}")

        truck_routes_for_day = [route for route, day in truck_routes if day == current_day]
        num_trucks_for_day = len(truck_routes_for_day)
        if num_trucks_for_day > 0:
            number_of_truck_days += 1
        truck_usage_count += num_trucks_for_day  # Ensure this is inside the loop to count trucks per day
        if num_trucks_for_day > max_trucks_used:
            max_trucks_used = num_trucks_for_day
        day_output.append(f"NUMBER_OF_TRUCKS = {num_trucks_for_day}")
        for truck_index, route in enumerate(truck_routes_for_day, start=1):
            route_output = f"{truck_index} " + ' '.join(map(str, route))
            day_output.append(route_output)

        technician_requests_for_day = {tech: reqs for tech, reqs in assigned_requests.items() if any(day == current_day for _, day in reqs)}
        num_technicians_for_day = len(technician_requests_for_day)
        if num_technicians_for_day > 0:
            number_of_technician_days += 1
        technician_usage_count += num_technicians_for_day  # Ensure this is inside the loop to count technicians per day
        if num_technicians_for_day > max_technicians_used:
            max_technicians_used = num_technicians_for_day
        day_output.append(f"NUMBER_OF_TECHNICIANS = {num_technicians_for_day}")
        for tech_id, requests in technician_requests_for_day.items():
            tech_requests_for_day = [req for req, day in requests if day == current_day]
            if tech_requests_for_day:
                day_output.append(f"{tech_id} " + ' '.join(map(str, tech_requests_for_day)))

        daily_output.append('\n'.join(day_output))

    IDLE_MACHINE_COST = total_idle_costs
    total_truck_distance_cost = total_truck_distance * TRUCK_DISTANCE_COST
    total_truck_day_cost = truck_usage_count * TRUCK_DAY_COST
    total_truck_cost = max_trucks_used * TRUCK_COST
    total_technician_distance_cost = total_distance_all_techs * TECHNICIAN_DISTANCE_COST
    total_technician_day_cost = technician_usage_count * TECHNICIAN_DAY_COST
    total_technician_cost = len(unique_technicians_used) * TECHNICIAN_COST  # Use unique_technicians_used for cumulative count

    total_cost = (total_truck_distance_cost + total_truck_day_cost + total_truck_cost +
                  total_technician_distance_cost + total_technician_day_cost + total_technician_cost + IDLE_MACHINE_COST)

    with open(output_file_name, "w") as file:
        file.write(f"DATASET = ORTEC Caroline VeRoLog 2019\n")
        file.write(f"NAME = {title}\n")
        file.write(f"TRUCK_DISTANCE = {int(total_truck_distance)}\n")
        file.write(f"NUMBER_OF_TRUCK_DAYS = {truck_usage_count}\n")
        file.write(f"NUMBER_OF_TECHNICIAN_DAYS = {technician_usage_count}\n")
        file.write(f"TECHNICIAN_DISTANCE = {int(total_distance_all_techs)}\n")
        file.write(f"NUMBER_OF_TRUCKS_USED = {max_trucks_used}\n")
        file.write(f"NUMBER_OF_TECHNICIANS_USED = {len(unique_technicians_used)}\n")  # Use unique_technicians_used for cumulative count
        file.write(f"IDLE_MACHINE_COSTS = {IDLE_MACHINE_COST}\n")
        file.write(f"TOTAL_COST = {int(total_cost)}\n")

        for output in daily_output:
            file.write("\n")
            file.write(output)
            file.write("\n")

    return (truck_usage_count, technician_usage_count, max_trucks_used, len(unique_technicians_used), total_cost)

truck_usage_count, technician_usage_count, max_trucks_used, cumulative_technicians_used, total_cost = print_schedule(
    truck_routes_extracted, delivery_days, assigned_requests, total_truck_distance, tech_route_distances_per_day, output_file_name, unique_technicians_used
)

print(f"TRUCK_DISTANCE = {int(total_truck_distance)}")
print(f"NUMBER_OF_TRUCK_DAYS = {truck_usage_count}")
print(f"NUMBER_OF_TECHNICIAN_DAYS = {technician_usage_count}")
print(f"TECHNICIAN_DISTANCE = {int(total_distance_all_techs)}")
print(f"NUMBER_OF_TRUCKS_USED = {max_trucks_used}")
print(f"NUMBER_OF_TECHNICIANS_USED = {cumulative_technicians_used}")
print(f"TOTAL_COST = {int(total_cost)}")


TRUCK_DISTANCE = 12090
NUMBER_OF_TRUCK_DAYS = 29
NUMBER_OF_TECHNICIAN_DAYS = 25
TECHNICIAN_DISTANCE = 7400
NUMBER_OF_TRUCKS_USED = 7
NUMBER_OF_TECHNICIANS_USED = 8
TOTAL_COST = 195610573
